In [ ]:
%pip install librosa tensorflow scikit-learn matplotlib

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

In [ ]:
!unzip gtzan-dataset-music-genre-classification.zip

In [ ]:
import os

data_path = "/content/Data/genres_original"
genres = os.listdir(data_path)
print("Genres:", genres)

# List some files from one genre
print("\nSample files from 'jazz':")
print(os.listdir(os.path.join(data_path, 'jazz'))[:5])

In [ ]:
#Preprocess the audio into MFCCs and save as data.json
import math
import json
import librosa
import numpy as np

DATASET_PATH = "/content/Data/genres_original"
JSON_PATH = "data.json"
SAMPLES_PER_TRACK = 22050 * 30

def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=10):
    data = {
        "mapping": [],
        "mfcc": [],
        "labels": []
    }

    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    expected_num_mfcc_vectors = math.ceil(num_samples_per_segment / hop_length)

    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        if dirpath != dataset_path:
            genre_label = dirpath.split("/")[-1]
            data["mapping"].append(genre_label)
            print(f"\nProcessing: {genre_label}")

            for f in filenames:
                file_path = os.path.join(dirpath, f)
                try:
                    signal, sr = librosa.load(file_path, sr=22050)
                except Exception as e:
                    print(f"Could not process file {file_path}: {e}")
                    continue


                for d in range(num_segments):
                    start = num_samples_per_segment * d
                    finish = start + num_samples_per_segment

                    mfcc = librosa.feature.mfcc(y=signal[start:finish], sr=sr,
                                                n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                    mfcc = mfcc.T

                    if len(mfcc) == expected_num_mfcc_vectors:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i - 1)

    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

    print("\n✅ Saved MFCCs to", json_path)

# Run preprocessing
save_mfcc(DATASET_PATH, JSON_PATH)


In [ ]:
import json
import numpy as np

DATA_PATH = "data.json"

def load_data(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y

X, y = load_data(DATA_PATH)
print(f"Data loaded. Shape of X: {X.shape}, Shape of y: {y.shape}")

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# reshape for CNN input: (samples, height, width, channels)
X = X[..., np.newaxis]

# one-hot encode labels
y = to_categorical(y)

# split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Train shape: {X_train.shape}, Validation shape: {X_val.shape}, Test shape: {X_test.shape}")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='softmax'))  # 10 genres

model.summary()


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=30,
                    batch_size=32)


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"\nTest accuracy: {test_acc:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()
